# Clustering

Un clúster es una agrupación de objetos que comparten similitudes, y los objetos que pertenecen a diferentes clústeres presentan diferencias. Encontrar grupos en los datos es el objetivo principal de la agrupación en clústeres. 

> El objetivo es minimizar las diferencias dentro de cada grupo y maximizar las diferencias entre los clústeres.

La agrupación en clústeres consiste en dividir un conjunto de datos finito sin etiquetar en subconuntos de datos diferenciados que emergen de estreucturas subyacentes en los datos.

Sea 

$$
\mathbf{X}= \{\mathbf{x}_1,\mathbf{x}_2,\ldots,\mathbf{x}_i,\ldots,\mathbf{x}_N\}
$$

un conjunto de objetos de entrada donde $\mathbf{x}_i = \left(x_{i1},x_{i2},\ldots,x_{ij},\ldots,x_{id}\right) \in \mathbb{R}^d$, y  $x_{ij}$  es una característica (atributo, dimensión o variable). Una partición $K$ de $\mathbf{X}$, $C = \{C_1,\ldots,C_K\}$ ($K\leq N$) es una partición dura si:


- Cada clúster debe tener al menos un elemento: $C_i\neq \emptyset$, $i=1,\ldots,K$. 
- La unión de todos los clústers es el conjunto de entrada $\mathbf{X}$: $\bigcup_{i=1}^KC_i=\mathbf{X}$. 
- Si un objeto pertenece a un cluster, no puede pertenecer a otro cluster: $C_i\cap C_j = \emptyset$, $i,j=1,\ldots,K$ and $i\neq j$.




## K-means

> El algoritmo k-means es quizás el método de agrupación más utilizado. Después de haber sido estudiado durante varias décadas, sirve como base para muchas técnicas de agrupación más sofisticadas.

### Ejemplo 

Veamos un ejemplo de k-means mediante la generación de clústers aleatorios (`make_blobs`) y `k=3`.

**Preámbulo y datos**

In [1]:

#%matplotlib notebook
import numpy as np
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt

from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from shared_utilities import plot_labelled_scatter

X, y = make_blobs(random_state = 10)

kmeans = KMeans(n_init = 3,n_clusters = 3)
kmeans.fit(X)

plot_labelled_scatter(X, kmeans.labels_, ['Cluster 1', 'Cluster 2', 'Cluster 3'])

ModuleNotFoundError: No module named 'graphviz'

**Algoritmo**

**Entradas:** 
   - Conjunto de patrones en $\mathbb{R}^n$.
   - $k$ número de grupos a formar.

**Salidas:** 

   - Una partición del espacio de patrones, tal que, optimiza la varianza global.

**Inicio:** 

Asignar aleatoreamente un número, del 1 a $K$ a cada observación. Estos funcionan como asignaciones iniciales para las observaciones.

**Repetición**

  1. Para cada uno de los K conglomerados, calcular el centroide del conglomerado. El k-ésimo centroide del grupo es el vector de las $p$ medias de características para las observaciones en el k-ésimo grupo.
  2. Asignar cada observación al conglomerado cuyo centroide es el más cercano (donde más cercano se define utilizando la distancia euclidiana).
  
**Hasta** La última partición obtenida, (idéntica a la de la iteración
anterior ) es la respuesta final del algoritmo.


### Ejemplo: clasificación de frutas
Ejemplo que muestra k-medias para encontrar 4 conglomerados en el conjunto de datos de frutas. Tenga en cuenta que, en general, es importante escalar las características individuales antes de aplicar la agrupación en clústeres de k-medias.

In [ ]:
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from shared_utilities import plot_labelled_scatter
from sklearn.preprocessing import MinMaxScaler
import pandas as pd

uu = 'https://raw.githubusercontent.com/vmoprojs/DataLectures/refs/heads/master/fruit_data_with_colors.txt'
fruits = pd.read_table(uu)
X_fruits = fruits[['mass','width','height', 'color_score']].to_numpy()
y_fruits = fruits[['fruit_label']] - 1

X_fruits_normalized = MinMaxScaler().fit(X_fruits).transform(X_fruits)  

kmeans = KMeans(n_init = 4, random_state = 0,n_clusters = 4)
kmeans.fit(X_fruits_normalized)

plot_labelled_scatter(X_fruits_normalized, kmeans.labels_, 
                      ['Cluster 1', 'Cluster 2', 'Cluster 3', 'Cluster 4'])

In [ ]:
# https://scikit-learn-extra.readthedocs.io/en/latest/index.html
from sklearn_extra.cluster import KMedoids
kmedoides = KMedoids(n_clusters = 4, random_state = 0)
kmedoides.fit(X_fruits_normalized)

plot_labelled_scatter(X_fruits_normalized, kmedoides.labels_, 
                      ['Cluster 1', 'Cluster 2', 'Cluster 3', 'Cluster 4'])

## Clustering Jerárquico: Aglomerativo

Una desventaja potencial del agrupamiento K-medias es que requiere que **especifiquemos previamente** el número de agrupamientos K. 

El agrupamiento jerárquico es un enfoque alternativo que **no requiere** que nos comprometamos con una elección particular de K. 

El agrupamiento jerárquico tiene una ventaja adicional sobre el agrupamiento K-medias, ya que da como resultado una atractiva representación basada en árboles de las observaciones, llamada **dendrograma**.

**Algoritmo**

**Entradas:** 
   - Conjunto de patrones en cualquier espacio.

**Salidas:** 

   - Dendograma de agrupamiento

**Inicio:** 

   - Seleccionar una función de comparación entre patrones.
   - Calcular la Matriz Global de Semejanza/Diferencia entre grupos.

   
      

**Repetición**

   1. Identificar la menor diferencia entre grupos y fusionar dichos grupos en un nuevo grupo con etiqueta única.
   2. Actualizar la matriz de sem/dif, eliminando el renglón y columna de los grupos fusionados y agregando un nuevo renglón y columna para el nuevo grupo
   3. Registrar la fusión realizada en el dendrograma
  
**Hasta** Terminar cuando todos los patrones se encuentren agrupados en un solo grupo.


En Python, el método [AgglomerativeClustering](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.AgglomerativeClustering.html) nos permite ejecutar clustering jerárquico.

In [ ]:
from sklearn.datasets import make_blobs
from sklearn.cluster import AgglomerativeClustering
from shared_utilities import plot_labelled_scatter

X, y = make_blobs(random_state = 10)

cls = AgglomerativeClustering(n_clusters = 3)
cls_assignment = cls.fit_predict(X)

plot_labelled_scatter(X, cls_assignment, 
        ['Cluster 1', 'Cluster 2', 'Cluster 3'])

In [ ]:
X, y = make_blobs(random_state = 10, n_samples = 10)
plot_labelled_scatter(X, y, 
        ['Cluster 1', 'Cluster 2', 'Cluster 3'])
print(X)

**Creamos el dendograma**

In [ ]:
from scipy.cluster.hierarchy import dendrogram,linkage
plt.figure()
dendrogram(linkage(X, 'complete'))
plt.show()

-   Cada hoja del dendrograma representa una de las $n$ observaciones.

-   A medida que subimos por el árbol, algunas hojas comienzan a *fusionarse* en ramas. Estos corresponden a observaciones que son similares entre sí.

-   Cuanto antes (más abajo en el árbol) se produzcan fusiones, más similares serán los grupos de observaciones entre sí. Por otro lado, las observaciones que se fusionan más tarde (cerca de la parte superior del árbol) pueden ser bastante diferentes.


-   La altura de esta fusión, medida en el eje vertical, **indica cuán diferentes son las dos observaciones**. Por lo tanto, las observaciones que se fusionan en la parte inferior del árbol son bastante similares entre sí, mientras que las observaciones que se fusionan cerca de la parte superior del árbol tenderán a ser bastante diferentes.

### Funciones para fusionar grupos

<img src="images/10_fig1.png" width="500" height="300">

-   *Complete*: Disimilitud máxima entre conglomerados. Calcule todas las diferencias por pares entre las observaciones en el grupo $A$ y las observaciones en el grupo $B$, y **registre la mayor de estas diferencias**.

-   *Single*: Disimilitud mínima entre conglomerados. Calcule todas las diferencias por pares entre las observaciones en el grupo $A$ y las observaciones en el grupo $B$, y **registre la más pequeña de estas diferencias**. El enlace único puede dar como resultado conglomerados extendidos y finales en los que las observaciones individuales se fusionan una a la vez.

-   *Average*: Diferencia media entre grupos. Calcule todas las diferencias por pares entre las observaciones en el grupo $A$ y las observaciones en el grupo $B$, y **registre el promedio de estas diferencias**.

-   *Centroid*: Disimilitud entre el centroide para el grupo $A$ (un vector de medias de longitud $p$) y el centroide para el grupo $B$.


En **Python** usamos los métodos:

- [dendrogram](https://docs.scipy.org/doc/scipy/reference/generated/scipy.cluster.hierarchy.dendrogram.html): para elaborar el dendograma.

- [linkage](https://docs.scipy.org/doc/scipy/reference/generated/scipy.cluster.hierarchy.linkage.html): Realiza agrupamiento jerárquico/aglomerativo.

In [ ]:
plt.figure()
plt.title('Linkage: Complete')
dendrogram(linkage(X, 'complete'))
plt.show()

plt.figure()
plt.title('Linkage: Single')
dendrogram(linkage(X, 'single'))
plt.show()

plt.figure()
plt.title('Linkage: Average')
dendrogram(linkage(X, 'average'))
plt.show()

plt.figure()
plt.title('Linkage: Centroid')
dendrogram(linkage(X, 'centroid'))
plt.show()

## DBSCAN clustering

Es un algoritmo no supervisado que tienen por objetivo descubrir grupos a partir de formas arbitrarias de cualquier conjunto de datos y al mismo tiempo puede distinguir *noise points* que no forman parte de ningún grupo.

**Parámetros**

- Epsilon ($\epsilon$): define el radio de una *bola*.
- MinPts: Número mínimo de puntos requeridos dentro de la bola de radio $\epsilon$

**Definiciones**:

-  Bola Cerrada: $B_{\epsilon}(p)=B(p;\epsilon)=\{x\in X\mid d(x,p)\leq \epsilon\}$, donde $p$ es el centro y $\epsilon$ es el radio
-  *Core point*: Si, dentro de la bola, contiene un número de puntos mayor o igual a *MinPts*
-  *Border point*: Si, dentro de la bola, contiene al menos un *core point*
-  *Noise point*: Si, dentro de la bola, no contiene ningún *core point*

<img src="images/dbscan.jpg" width="500" height="300">

En Python:

- [`DBSCAN`](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.DBSCAN.html) es el método para ejecutar el algortimo.

In [ ]:
from sklearn.cluster import DBSCAN
from sklearn.datasets import make_blobs

X, y = make_blobs(random_state = 9, n_samples = 25)

dbscan = DBSCAN(eps = 2, min_samples = 2)

cls = dbscan.fit_predict(X)
print("Cluster membership values:\n{}".format(cls))

plot_labelled_scatter(X, cls + 1, 
        ['Noise', 'Cluster 0', 'Cluster 1', 'Cluster 2'])

### Ejemplo: clasificación de clientes

In [ ]:
uu = "https://raw.githubusercontent.com/vmoprojs/DataLectures/master/Mall_Customers.csv"
datos = pd.read_csv(uu)


dbDat = datos.iloc[:,3:5]
dbDat.describe()

In [ ]:
plt.figure()
plt.scatter(dbDat.AnnualIncome,dbDat.SpendingScore)
plt.show()

In [ ]:
from scipy.spatial import distance_matrix
dd = pd.DataFrame(distance_matrix(dbDat.values, dbDat.values), index=dbDat.index, columns=dbDat.index)
n = len(np.sort(dd.values))
dd = np.sort(dd.values[np.triu_indices(n, k = 1)])

plt.figure()
plt.plot(dd)
plt.show()

In [ ]:
dbscan = DBSCAN(eps = 10, min_samples = 3)

cls = dbscan.fit_predict(dbDat)
print("Cluster membership values:\n{}".format(cls))

plot_labelled_scatter(dbDat.values, cls + 1, 
        ['Noise', 'Cluster 0', 'Cluster 1', 'Cluster 2'])


**Un ejemplo combinado**

In [ ]:
www = "https://stat.ethz.ch/Teaching/Datasets/WBL/crime2.dat"
crime = pd.read_csv(www,sep=" ")
crime.describe()

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# Antes de aplicar ACP, cada variable debe ser centrada y de varianza igual a 1
X_normalized = StandardScaler().fit(crime).transform(crime) 

In [ ]:
pca = PCA(n_components = 2).fit(X_normalized)

X_pca = pca.transform(X_normalized) # nuevos componentes
print('varianza explicada',pca.explained_variance_ratio_) # vaianza explicada

grupos = KMeans(n_clusters = 3)
grupos.fit(X_pca)

plot_labelled_scatter(X_pca, grupos.labels_, ['Cluster 1', 'Cluster 2', 'Cluster 3'])


In [ ]:
dbscan = DBSCAN(eps = 0.8, min_samples = 3)

cls = dbscan.fit_predict(X_pca)
print(np.unique(cls))
plot_labelled_scatter(X_pca, cls + 1, 
        ['Noise', 'Cluster 0', 'Cluster 1', 'Cluster 2'])